In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import TimestampType

In [ ]:
payments = spark.table("default.staging_payments")
display(payments.limit(10))

payment_id,date,amount,rider_id
1,2019-05-01T00:00:00.000+0000,9.0,1000
2,2019-06-01T00:00:00.000+0000,9.0,1000
3,2019-07-01T00:00:00.000+0000,9.0,1000
4,2019-08-01T00:00:00.000+0000,9.0,1000
5,2019-09-01T00:00:00.000+0000,9.0,1000
6,2019-10-01T00:00:00.000+0000,9.0,1000
7,2019-11-01T00:00:00.000+0000,9.0,1000
8,2019-12-01T00:00:00.000+0000,9.0,1000
9,2020-01-01T00:00:00.000+0000,9.0,1000
10,2020-02-01T00:00:00.000+0000,9.0,1000


In [ ]:
# Check whether payment_id is duplicated
print(payments.count())
fact_payments = payments.dropDuplicates(["payment_id"])
print(fact_payments.count())

1946607
1946607

In [ ]:
riders = spark.table("default.staging_riders")

In [ ]:
# Check whether payment_id is duplicated
print(riders.count())
riders = riders.dropDuplicates(["rider_id"])
print(riders.count())

75000
75000

In [ ]:
riders.show()


+--------+-----------+---------+--------------------+-------------------+-------------------+-------------------+---------+
rider_id| first_name|last_name| address| birthday| account_start_date| account_end_date|is_member|
+--------+-----------+---------+--------------------+-------------------+-------------------+-------------------+---------+
 1000| Diana| Clark| 1200 Alyssa Squares|1989-02-13 00:00:00|2019-04-23 00:00:00| null| true|
 1001| Jennifer| Smith| 397 Diana Ferry|1976-08-10 00:00:00|2019-11-01 00:00:00|2020-09-01 00:00:00| true|
 1002| Karen| Smith|644 Brittany Row ...|1998-08-10 00:00:00|2022-02-04 00:00:00| null| true|
 1003| Bryan| Roberts|996 Dickerson Tur...|1999-03-29 00:00:00|2019-08-26 00:00:00| null| false|
 1004| Jesse|Middleton|7009 Nathan Expre...|1969-04-11 00:00:00|2019-09-14 00:00:00| null| true|
 1005| Christine|Rodriguez|224 Washington Mi...|1974-08-27 00:00:00|2020-03-24 00:00:00| null| false|
 1006| Alicia| Taylor| 1137 Angela Locks|2004-01-30 00:00:00|2020-11-27 00:00:00|2021-12-01 00:00:00| true|
 1007| Benjamin|Fernandez| 979 Phillips Ways|1988-01-11 00:00:00|2016-12-11 00:00:00| null| false|
 1008| John| Crawford| 7691 Evans Court|1987-02-21 00:00:00|2021-03-28 00:00:00|2021-07-01 00:00:00| true|
 1009| Victoria| Ritter|9922 Jim Crest Ap...|1981-02-07 00:00:00|2020-06-12 00:00:00|2021-11-01 00:00:00| true|
 1010| Tracy| Austin| 92973 Mary Ville|1996-04-07 00:00:00|2019-12-27 00:00:00| null| true|
 1011| Jessica| Mcgee|950 Grimes Burg A...|1984-12-29 00:00:00|2017-05-20 00:00:00| null| true|
 1012| Heather| Fisher|65532 Davis Sprin...|1980-10-20 00:00:00|2021-10-16 00:00:00| null| true|
 1013| Timothy| Jones| 7757 Johnston Roads|1985-07-10 00:00:00|2020-12-28 00:00:00|2021-11-01 00:00:00| true|
 1014| Jennifer| Martin| 501 Arellano Land|1989-12-04 00:00:00|2017-11-24 00:00:00| null| true|
 1015|Christopher| Silva|3710 Rodriguez Gl...|2001-07-25 00:00:00|2017-07-10 00:00:00| null| true|
 1016| Andrew| Jones| 72226 Casey Square|1991-12-13 00:00:00|2022-02-02 00:00:00| null| true|
 1017| William| Lawson| 40395 Terrell Parks|1981-04-17 00:00:00|2019-03-11 00:00:00| null| true|
 1018| Shelly| Briggs| 3514 Leslie Vista|1986-09-02 00:00:00|2021-07-25 00:00:00| null| false|
 1019| Tina| Garcia|00348 Brandi Park...|1997-05-03 00:00:00|2021-07-10 00:00:00|2022-01-01 00:00:00| false|
+--------+-----------+---------+--------------------+-------------------+-------------------+-------------------+---------+
only showing top 20 rows

In [ ]:
# age = riders.select(
#     datediff(("birthday", "account_start_date")).cast('int').alias('age')  
# )
age = riders.withColumn('age', (months_between(current_date(), col("birthday")) / 12).cast('int'))
# age = riders.withColumn("diff_in_years", datediff(col("account_start_date"),col("birthday"))/365.25)
age.show()


+--------+----------+---------+--------------------+-------------------+-------------------+-------------------+---------+---+
rider_id|first_name|last_name| address| birthday| account_start_date| account_end_date|is_member|age|
+--------+----------+---------+--------------------+-------------------+-------------------+-------------------+---------+---+
 1005| Christine|Rodriguez|224 Washington Mi...|1974-08-27 00:00:00|2020-03-24 00:00:00| null| false| 47|
 1008| John| Crawford| 7691 Evans Court|1987-02-21 00:00:00|2021-03-28 00:00:00|2021-07-01 00:00:00| true| 35|
 1016| Andrew| Jones| 72226 Casey Square|1991-12-13 00:00:00|2022-02-02 00:00:00| null| true| 30|
 1019| Tina| Garcia|00348 Brandi Park...|1997-05-03 00:00:00|2021-07-10 00:00:00|2022-01-01 00:00:00| false| 25|
 1025| Bill| Gregory|5182 Michelle Pla...|2004-12-06 00:00:00|2020-05-01 00:00:00| null| false| 17|
 1030| Jeremy| Roberts|2722 Kaitlin Stre...|2000-05-13 00:00:00|2021-02-09 00:00:00| null| true| 22|
 1031| Eric| Kelly| 47822 Darrell Green|1986-07-04 00:00:00|2018-08-16 00:00:00| null| true| 36|
 1034| Christy| Cobb| 9001 Carrie Alley|1988-12-30 00:00:00|2020-03-08 00:00:00| null| false| 33|
 1046| Brianna| Flores| 351 Bill Curve|2005-11-09 00:00:00|2020-07-08 00:00:00| null| true| 16|
 1047| Kimberly| Delgado|2639 Ronald Summi...|1988-10-29 00:00:00|2021-02-23 00:00:00|2021-03-01 00:00:00| true| 33|
 1051| Tony| Anderson|19355 Corey Run A...|1981-07-25 00:00:00|2021-01-14 00:00:00| null| false| 41|
 1055| Jamie| Jones|2345 Elizabeth Vi...|1993-09-27 00:00:00|2021-09-22 00:00:00| null| false| 28|
 1056| Heidi| Ramos|81917 Jonathan Fo...|1983-10-19 00:00:00|2014-12-05 00:00:00| null| false| 38|
 1061| Mark| Watson|7524 Ferrell Junc...|1976-05-15 00:00:00|2021-04-27 00:00:00| null| true| 46|
 1064| Julia| Smith| 023 Larry Knolls|1986-08-03 00:00:00|2021-03-25 00:00:00| null| true| 36|
 1065| Rebecca|Mccormick|4912 Smith Alley ...|1983-07-18 00:00:00|2017-07-13 00:00:00| null| false| 39|
 1068| Charles| Garza|0075 Sean Meadow ...|1987-12-24 00:00:00|2022-01-04 00:00:00| null| false| 34|
 1077| Hailey| Green|210 Pearson Exten...|1966-09-20 00:00:00|2021-12-05 00:00:00|2021-12-01 00:00:00| true| 55|
 1084| Nathaniel| Johnson|80941 Brittany Ke...|1991-01-25 00:00:00|2019-11-30 00:00:00| null| true| 31|
 1085| Jesse| White|5864 Jennings For...|1992-12-02 00:00:00|2019-02-05 00:00:00| null| true| 29|
+--------+----------+---------+--------------------+-------------------+-------------------+-------------------+---------+---+
only showing top 20 rows

In [ ]:
fact_payments = fact_payments.join(age,'rider_id').select(fact_payments['*'], age['age'].alias('rider_age'))

In [ ]:
fact_payments.show()

+----------+-------------------+------+--------+---------+
payment_id| date|amount|rider_id|rider_age|
+----------+-------------------+------+--------+---------+
 148|2017-09-01 00:00:00| 6.86| 1007| 34|
 463|2021-09-01 00:00:00| 22.35| 1018| 35|
 471|2021-10-01 00:00:00| 12.52| 1019| 25|
 496|2021-11-01 00:00:00| 9.0| 1020| 37|
 833|2021-04-01 00:00:00| 9.0| 1035| 19|
 1088|2019-10-01 00:00:00| 9.0| 1041| 53|
 1238|2020-10-01 00:00:00| 9.0| 1043| 35|
 1342|2021-12-01 00:00:00| 9.0| 1049| 28|
 1580|2018-04-01 00:00:00| 15.05| 1056| 38|
 1591|2019-03-01 00:00:00| 17.22| 1056| 38|
 1645|2016-04-01 00:00:00| 13.3| 1057| 34|
 1829|2021-01-01 00:00:00| 7.39| 1065| 39|
 1959|2019-05-01 00:00:00| 21.78| 1072| 28|
 2122|2020-10-01 00:00:00| 9.0| 1082| 19|
 2142|2020-08-01 00:00:00| 9.0| 1083| 40|
 2366|2017-02-01 00:00:00| 9.0| 1097| 43|
 2659|2018-01-01 00:00:00| 7.83| 1101| 38|
 2866|2015-10-01 00:00:00| 9.0| 1108| 35|
 3175|2019-03-01 00:00:00| 15.76| 1124| 27|
 3749|2020-08-01 00:00:00| 9.0| 1140| 25|
+----------+-------------------+------+--------+---------+
only showing top 20 rows

In [ ]:
fact_payments.write.format("delta").mode("overwrite").saveAsTable("default.fact_payment")